This notebook shows a method to crop the T-Maze arenas from Alice and Samara's semester project experiments

# Packages imports

In [ ]:
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path
import re
from tqdm import tqdm
import cv2
import numpy as np
from pathlib import Path
from scipy import signal
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
import mpldatacursor
import shutil
from itertools import repeat
import subprocess
import sys
import os
from joblib import Parallel, delayed

# Opening and visualizing the sample image

In [ ]:
img = cv2.imread("image35682.jpg")

if len(img.shape) > 2:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
fig, ax = plt.subplots()
plt.axis("off")
im = ax.imshow(img, cmap="gray", vmin=0, vmax = 255)

mpldatacursor.datacursor()

plt.show()



# Cropping the arenas

In [ ]:
import cv2
import matplotlib.pyplot as plt

X1 = 0
X2 = 620
X3 = 1450
X4 = 2130
X5 = 2980
X6 = 3590

Y1 = 0
Y2 = 725
Y3 = 1140
Y4 = 1860
Y5 = 2350
Y6 = 2995

regions_of_interest = [
    (X1, Y1, X2, Y2),
    (X3, Y1, X4, Y2),
    (X5, Y1, X6, Y2),
    (X1, Y3, X2, Y4),
    (X3, Y3, X4, Y4),
    (X5, Y3, X6, Y4),
    (X1, Y5, X2, Y6),
    (X3, Y5, X4, Y6),
    (X5, Y5, X6, Y6),
]

fig, axs = plt.subplots(3, 3, figsize=(20, 20))
for i in range(9):
    axs[i // 3, i % 3].axis("off")
    axs[i // 3, i % 3].imshow(
        img[
            regions_of_interest[i][1] : regions_of_interest[i][3],
            regions_of_interest[i][0] : regions_of_interest[i][2],
        ],
        cmap="gray",
        vmin=0,
        vmax=255,
    )

# Remove the axis of each subplot and draw them closer together
for ax in axs.flat:
    ax.axis("off")
plt.subplots_adjust(wspace=0, hspace=0)

In [ ]:
Arena1 = img[
    regions_of_interest[0][1] : regions_of_interest[0][3],
    regions_of_interest[0][0] : regions_of_interest[0][2],
]

# Find contours in the arena

In [ ]:
# Apply histogram equalization to mitigate brightness variations

frame = cv2.equalizeHist(Arena1)
# Apply a binary threshold to the image
_, threshold = cv2.threshold(Arena1, 80, 255, cv2.THRESH_BINARY)

# Do some erode and dilate to remove noise
kernel = np.ones((5, 5), np.uint8)

#threshold = cv2.erode(threshold, kernel, iterations=1)

threshold = cv2.dilate(threshold, kernel, iterations=1)

# Display the image

fig, ax = plt.subplots()
plt.axis("off")
im = ax.imshow(threshold, cmap="gray", vmin=0, vmax=255)

plt.show()

In [ ]:

# Find contours in the thresholded image
contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Plot the contours on the original image
fig, ax = plt.subplots()
plt.axis("off")
im = ax.imshow(Arena1, cmap="gray", vmin=0, vmax=255)
for contour in contours:
    ax.plot(contour[:, 0, 0], contour[:, 0, 1], color="red")
    
plt.show()

In [ ]:
# Find most outer contours

# Find the 3 contours with the largest area

contours = sorted(contours, key=cv2.contourArea, reverse=True)[:3]

# Plot the contours on the original image

fig, ax = plt.subplots()
plt.axis("off")
im = ax.imshow(Arena1, cmap="gray", vmin=0, vmax=255)

for contour in contours:
    ax.plot(contour[:, 0, 0], contour[:, 0, 1], color="red")

plt.show()

In [ ]:
# Crop the image to the bounding box of the three contours to generate 3 T mazes

# Get the bounding box of the 3 contours

bounding_boxes = [cv2.boundingRect(contour) for contour in contours]

# Crop the image to the bounding box of the 3 contours

T_mazes = [
    Arena1[y : y + h, x : x + w] for x, y, w, h in bounding_boxes
]

# Display the 3 T mazes

fig, axs = plt.subplots(1, 3, figsize=(20, 20))
for i in range(3):
    axs[i].axis("off")
    axs[i].imshow(T_mazes[i], cmap="gray", vmin=0, vmax=255)
    
plt.show()

In [ ]:
# Get the bounding box of the 3 contours
bounding_boxes = [cv2.boundingRect(contour) for contour in contours]

T_mazes = []

for contour, (x, y, w, h) in zip(contours, bounding_boxes):
    # Create a black image with the same dimensions as the original
    mask = np.zeros_like(Arena1)

    # Draw the contour on the mask in white
    cv2.drawContours(mask, [contour], -1, (255), thickness=cv2.FILLED)

    # Apply the mask to the image
    masked_image = cv2.bitwise_and(Arena1, mask)

    # Crop the image to the bounding box of the contour
    T_mazes.append(masked_image[y : y + h, x : x + w])

# Display the 3 T mazes
fig, axs = plt.subplots(1, 3, figsize=(20, 20))
for i in range(3):
    axs[i].axis("off")
    axs[i].imshow(T_mazes[i], cmap="gray", vmin=0, vmax=255)

plt.show()